<a href="https://colab.research.google.com/github/ghosh-sarbajit/DifferentialPrivacy/blob/main/DPSgd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install opacus
!pip install opendp
from IPython.display import clear_output
clear_output()

In [6]:
import numpy as np
import opendp.prelude as dp

In [7]:
def make_nabla_loss_i(w):
    dp.assert_features("contrib", "floating-point")
    w_0, w_1 = w
    def f_compute_grads(data):
        x, y = data[np.newaxis].T
        y_hat = w_0 + w_1 * x # forward pass y^ = f(x)
        return (y_hat - y) * np.column_stack([np.ones(x.size), x])
    space = dp.np_array2_domain(T=float), dp.symmetric_distance()
    return dp.t.make_user_transformation(
        *space, *space, f_compute_grads,
        stability_map=lambda b_in: b_in)

In [3]:
import pandas as pd
import torch
import torch.nn as nn
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import LabelEncoder, StandardScaler, \
OrdinalEncoder, Normalizer
from opacus import PrivacyEngine

In [ ]:
class AdultDataSet(Dataset):
    def __init__(self, adult_data_file):
    header = ['age', 'workclass', 'fnlwgt', 'education', 'education_num',
    'marital_status', 'occupation', 'relationship',
    'race', 'sex', 'capital_gain', 'capital_loss',
    'hours_per_week', 'native_country', 'income']
    df = pd.read_csv(adult_data_file, header=None, names=header,
    sep=',\\s', na_values=['?'], engine='python')
    df = df.dropna()
    df = df.reset_index(drop=True)
    df['income'] = df['income'].apply(lambda x: x.replace('.', ''))
    categorical_columns = ['workclass', 'education', 'marital_status',
    'occupation', 'relationship', 'race', 'sex',
    'native_country']
    numerical_columns = ['age', 'capital_gain',
    'capital_loss', 'hours_per_week']
    column_transformer = make_column_transformer(
    (OrdinalEncoder(), categorical_columns),
    (StandardScaler(), numerical_columns),
    )
    self.y = LabelEncoder().fit_transform(df['income']).astype(float)
    self.X = column_transformer.fit_transform(df)
    self.X = Normalizer().fit_transform(self.X)
    def __len__(self):
    return len(self.y)